In [1]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    # 建立連接
    client = MongoClient('mongodb://localhost:27017/')
    
    # 檢查連接
    server_info = client.server_info()
    print("成功連接到 MongoDB!")
    print(f"MongoDB 版本: {server_info['version']}")
    
except ConnectionFailure as e:
    print(f"無法連接到 MongoDB 服務器: {e}")


成功連接到 MongoDB!
MongoDB 版本: 8.0.5


In [3]:
import pandas as pd

# 基本讀取
# 如果有中文，建議使用 utf-8 編碼
df = pd.read_csv('gmap838_reviews (1).csv', encoding='utf-8')

In [5]:
# 查看基本信息
print(df.info())

# 查看欄位名稱
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6378475 entries, 0 to 6378474
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   user_name            object
 1   user_id              object
 2   review_id            object
 3   score                int64 
 4   date                 object
 5   comments             object
 6   language             object
 7   translated_comments  object
 8   user_page            object
 9   gmap_location        object
dtypes: int64(1), object(9)
memory usage: 486.6+ MB
None
Index(['user_name', 'user_id', 'review_id', 'score', 'date', 'comments',
       'language', 'translated_comments', 'user_page', 'gmap_location'],
      dtype='object')


In [7]:
print(df['score'][0])

4


In [9]:
import numpy as np
import pandas as pd

def convert_numpy_types(item):
    """
    將 NumPy 和 Pandas 類型轉換為 Python 原生類型
    """
    if isinstance(item, dict):
        return {k: convert_numpy_types(v) for k, v in item.items()}
    elif isinstance(item, list):
        return [convert_numpy_types(i) for i in item]
    elif isinstance(item, np.integer):
        return int(item)
    elif isinstance(item, np.floating):
        # 處理 np.nan
        return float(item) if not np.isnan(item) else None
    elif isinstance(item, np.ndarray):
        return item.tolist()
    elif pd.isna(item):  # 處理 pandas NA/NaN
        return None
    else:
        return item

# 示例用法
user = [ {
    "name" : df['user_name'][0],
    "user_id" : df['user_id'][0],
    "review_id" : df['review_id'][0],
    "score" : df['score'][0],
    "date" : df['date'][0],
    "comments" : df['comments'][0],
    'language' : df['language'][0],
    'translated_comments' : df['translated_comments'][0],
    'user_page' : df['user_page'][0] , 
    'gmap_location' : df['gmap_location'][0]
}
]

# 轉換數據
converted_user = convert_numpy_types(user)

# 現在 converted_user 包含的是 Python 原生類型
# score 將是 int 類型而不是 np.int64
# translated_comments 將是 None 而不是 np.nan


In [11]:
print(converted_user)

[{'name': '林哲豪', 'user_id': '107033607967130169527', 'review_id': 'ChZDSUhNMG9nS0VJQ0FnTURBMUttdUZnEAE', 'score': 4, 'date': '2025-02-07', 'comments': '來參觀陶瓷博物館，可以了解台灣陶瓷的演進史，和現代陶瓷應用，讓我增加對陶瓷的知識', 'language': 'zh-Hant', 'translated_comments': None, 'user_page': 'https://www.google.com/maps/contrib/107033607967130169527/reviews?hl=zh-TW', 'gmap_location': '新北市立鶯歌陶瓷博物館'}]


In [13]:
# 選擇或創建數據庫
db = client['test']  # 如果不存在會在插入數據時自動創建
# 選擇或創建集合
collection = db['users']  # 如果不存在會在插入數據時自動創建

result = collection.insert_many(converted_user)
print(f"已插入文檔，ID列表: {result.inserted_ids}")

已插入文檔，ID列表: [ObjectId('67bec8fdf088484bbbdc9b59')]


In [4]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
# 使用 IP 地址而不是 localhost
client = MongoClient('mongodb://127.0.0.1:27017/')

# 或者指定更多連接參數
client = MongoClient(
    host='localhost',
    port=27017,
    serverSelectionTimeoutMS=5000  # 縮短超時時間以更快獲得錯誤
)
